# Ejercicio análisis time series
Para este ejercicio vas a trabajar con el histórico de valores en bolsa de una empresa que se dedica a vender muebles. Se pide
1. Importa los datos 'df_furniture.csv'
2. Trata la fecha como texto. Crea una columna para el año, mes y dia, y a partir de esas columnas crea la de fecha, en formato datetime de pandas
3. ¿De qué fecha a qué fecha va el histórico?
4. ¿Cómo es la serie temporal? ¿Tiene tendencia? ¿Aprecias alguna estacionalidad?
5. ¿Podrías demostrar de dos maneras estadísticas que la serie no es stationary?
6. Parece que hay una tendencia, pero no se aprecia bien con tanto pico en la serie. ¿Conoces algún método para representar mejor la tendencia?
7. Estaría bien visualizar todos los componentes de la serie temporal por separado
8. Demuestra mediante una gráfica de autocorrelación que la serie es estacional
9. ¿Qué dependencia tiene cada instante de sus ultimos 5 lags? Demuéstralo mediante una matriz de correlación. ¿Esto es bueno para los modelos?
10. Ya hemos visto que hay seasonality. ¿Cada cuántos instantes se cumple ese patrón?
11. Para un futuro modelado de los datos necesitaremos eliminar la tendencia. ¿Cómo hacemos esto?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Cargamos los datos

In [ ]:
df = pd.read_csv('data/df_furniture.csv', parse_dates=['Month'])
df

In [ ]:
# df['Month'] = pd.to_datetime(df['Month'])

In [ ]:
df.info()

In [ ]:
df[df['Month'].isna()]

In [ ]:
len(df)

In [ ]:
df['Month'].min()

In [ ]:
df['Month'].max()

In [ ]:
all_dates = pd.date_range(df['Month'].min(),df['Month'].max(),freq='MS')

In [ ]:
df['Month'] = all_dates
df

In [ ]:
df.info()

## 2. Tratar la fecha como texto y parsear

In [ ]:
df2 = df.copy()

df2['date_str'] = df2['Month'].astype(str)
df2[['year','month','day']] = df2['date_str'].str.split('-', expand=True)


In [ ]:
df2[['year','month','day']] = df2[['year','month','day']].astype(int)

In [ ]:
df2.info()

In [ ]:
df2['new_date'] = pd.to_datetime(df2[['year','month','day']])
df2

## 3. ¿De qué fecha a qué fecha va el histórico?

La serie va desde enero de 1992 a julio de 2019

In [ ]:
df.rename(columns={"Month":"Date"}, inplace=True)

In [ ]:
df

In [ ]:
print(df['Date'].min())
print(df['Date'].max())

In [ ]:
(df['Date'].max()-df['Date'].min()).days/365.25

## 4. ¿Cómo es la serie temporal? ¿Tiene tendencia? ¿Aprecias alguna estacionalidad?

Se ve que tiene una clara tendencia al alza, a pesar de una pequeña bajada en als acciones en el 2007.

También apreciamos estacionalidad, al cumplirse cierto patrón a lo largo del tiempo

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df.plot()

## 5. ¿Podrías demostrar de dos maneras estadísticas que la serie no es stationary?

In [ ]:
df

In [ ]:
#!pip install statsmodels

In [ ]:
from statsmodels.tsa.stattools import adfuller

adfuller(df.values)[1]

In [ ]:
df.describe()

In [ ]:
int(len(df)/2)

In [ ]:
df[:int(len(df)/2)].describe()

In [ ]:
df[int(len(df)/2):].describe()

## 6. ¿Conoces algún método para representar mejor la tendencia?

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess
lowess(df['furniture_price_adjusted'].values,np.arange(len(df)),frac=0.1)

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

df_lowess = pd.DataFrame(lowess(df['furniture_price_adjusted'].values, np.arange(len(df)),frac=0.1)[:,1], index=df.index, columns=['value'])
df_lowess.plot()

## 7. Estaría bien visualizar todos los componentes de la serie temporal por separado

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result_mul = seasonal_decompose(df['furniture_price_adjusted'], model = "multiplicative")
result_add = seasonal_decompose(df['furniture_price_adjusted'], model = "additive")
plt.style.use("default")
result_add.plot();

In [ ]:
result_mul.plot();

## 8. Demuestra mediante una gráfica de autocorrelación que la serie es estacional

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(df['furniture_price_adjusted']);
# plt.xticks()
# plt.xlim()

## 9. ¿Qué dependencia tiene cada instante de sus ultimos 5 lags? Demuestralo mediante una matriz de correlación.

In [ ]:
df2 = df.copy()

df2['lag1'] = df2['furniture_price_adjusted'].shift(1)
df2['lag2'] = df2['furniture_price_adjusted'].shift(2)
df2['lag3'] = df2['furniture_price_adjusted'].shift(3)
df2['lag4'] = df2['furniture_price_adjusted'].shift(4)
df2['lag5'] = df2['furniture_price_adjusted'].shift(5)
df2.head(20)

In [ ]:
df2.corr()

In [ ]:
import seaborn as sns
sns.heatmap(df2.corr(), annot=True)

## 10. Ya hemos visto que hay seasonality. ¿Cada cuántos instantes se cumple ese patrón?

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(df['furniture_price_adjusted'], lags=50);

## 11. Para un futuro modelado de los datos necesitaremos eliminar la tendencia. ¿Cómo hacemos esto?

In [ ]:
df

In [ ]:
result_add = seasonal_decompose(df['furniture_price_adjusted'], model = "additive", extrapolate_trend='freq')
detrended = df['furniture_price_adjusted'].values - result_add.trend

In [ ]:
detrended.plot()

In [ ]:
diff = df['furniture_price_adjusted'].diff(1).dropna()
diff.plot()